In [1]:
input_file = "day05/input"
lines = [l.strip() for l in open(input_file, "r").readlines()]

In [2]:
import more_itertools
groupped = list(more_itertools.split_at(lines, lambda x: x == ''))
seeds = groupped[0][0]
mappings = groupped[1:]

seeds = [int(x.strip()) for x in seeds.split(':')[1].strip().split(' ')]

In [3]:
import collections
import re

Range = collections.namedtuple("Range", ["target_start", "source_start", "length"])
Mapping = collections.namedtuple("Mapping", ["source", "target", "ranges"])

mapping_r = re.compile("([a-z]+)-to-([a-z]+)")
def parse_mapping(lines):
    def get_range(line):
        return Range(*map(int, line.split(' ')))
    header = mapping_r.match(lines[0])
    ranges = list(map(get_range, lines[1:]))
    return Mapping(header.group(1), header.group(2), ranges)

mappings = dict((m.source, m) for m in map(parse_mapping, mappings))

In [4]:
def find_location(seed):
    source = "seed"
    target = None
    value = seed

    while target != "location":
        target_mapping = mappings[source]
        target = target_mapping.target
        source = target
        mr = [r for r in target_mapping.ranges if r.source_start <= value < r.source_start + r.length]
        value = mr[0].target_start + value - mr[0].source_start if mr else value
    return value

print(f"Part 1: {min(map(find_location, seeds))}")

Part 1: 510109797


In [5]:
class Interval(object): # start inclusive, end exclusive
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def overlap(self, other): # ret: Interval?, [Interval] - first is overlap (if any), second "holes"
        if other.end <= self.start or other.start >= self.end:
            return (None, [self])
        o = Interval(max(self.start, other.start), min(self.end, other.end))
        h = []
        if self.end > other.end:
            h = h + [Interval(o.end, self.end)]
        if self.start < other.start:
            h = h + [Interval(self.start, other.start)]
        return (o, h)

    def __repr__(self):
        return f"[{self.start}, {self.end})"

def apply_mapping(interval, mapping): # yields 1+ mapped intervals
    unmappeds = [interval]
    mapped = []
    for r in mapping.ranges:
        nu = []
        for unmapped in unmappeds:
            m, h = unmapped.overlap(Interval(r.source_start, r.source_start + r.length))
            if m:
                delta = r.target_start - r.source_start
                mapped.append(Interval(m.start + delta, m.end + delta))
            nu = nu + h
        unmappeds = nu
    return mapped + unmappeds
            
import functools
def find_location2(seed_interval):
    source = "seed"
    target = None
    value = [seed_interval]

    while target != "location":
        target_mapping = mappings[source]
        target = target_mapping.target
        source = target

        value = functools.reduce(lambda x, y: x + y, [apply_mapping(i, target_mapping) for i in value], [])
    location = min(map(lambda interval: interval.start, value))
    return location

import itertools
seeds = list(itertools.starmap(lambda start, length: Interval(start, start + length), more_itertools.sliced(seeds, 2)))
print(f"Part 2: {min(map(find_location2, seeds))}")


Part 2: 9622622
